In [1]:
import sys
sys.path.insert(1, '../..')
from utils import *
import cv2
import convml_tt
from convml_tt.system import TripletTrainerModel, TripletTrainerDataModule
from convml_tt.data.examples import fetch_example_dataset, ExampleData, fetch_pretrained_model, PretrainedModel
from convml_tt.data.dataset import TileType, ImageSingletDataset
from convml_tt.utils import get_embeddings
import convml_tt.interpretation
import cv2
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
import numpy as np
import xarray as xr
import pandas as pd
import pytorch_lightning as pl
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
import torch
from sklearn.decomposition import PCA
from PIL import Image
import os, shutil
from matplotlib.patches import Rectangle as rectan
from random import seed
from random import randint
from sklearn.metrics import confusion_matrix
from PIL import Image
import matplotlib.pyplot as plt
from PIL import Image
from matplotlib import cm
from convml_tt.interpretation.rectpred.transform import apply_transform
from convml_tt.data.examples import load_pretrained_model, PretrainedModel
from convml_tt.interpretation.rectpred.data import make_sliding_tile_model_predictions
from convml_tt.interpretation.rectpred.plot import make_rgb
from tqdm.notebook import tqdm
from math import sqrt, cos, sin
import random
import operator

TILE_FILENAME_FORMAT = "{triplet_id:05d}_{tile_type}.png"
TEST_SIZE = 0.3
RANDOM_STATE = 1024
COLORS = ['b', 'g', 'r', 'm'] # Color of each class
#DATASETS_path = "../../../DATASETS/"
#DATASET_DIR = "../../../DATASETS/zooniverse_kaggle/"
DATASETS_path = "/home/fbrient/Dropbox/Documents/EUREC4A/Data/"
DATASET_DIR = DATASETS_path+"Kaggle/"
print(cv2.__version__)

pca = PCA(n_components=3, svd_solver='arpack')

4.5.3
4.5.3


In [2]:
#df = pd.read_csv("../../../DATASETS/zooniverse_kaggle/train.csv")
df = pd.read_csv(DATASET_DIR+"train.csv")

/home/fbrient/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
df['Image'] = df['Image_Label'].map(lambda x: x.split('_')[0])
df['Class'] = df['Image_Label'].map(lambda x: x.split('_')[1])
classes = df['Class'].unique()
train_df = df.groupby('Image')['Class'].agg(set).reset_index()
for class_name in classes:
    df[class_name] = df['Class'].map(lambda x: 1 if class_name in x else 0)
df.head()

/home/fbrient/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Image_Label,EncodedPixels,Image,Class,Fish,Flower,Gravel,Sugar
0,0011165.jpg_Fish,264918 937 266318 937 267718 937 269118 937 27...,0011165.jpg,Fish,1,0,0,0
1,0011165.jpg_Flower,1355565 1002 1356965 1002 1358365 1002 1359765...,0011165.jpg,Flower,0,1,0,0
2,0011165.jpg_Gravel,NaN,0011165.jpg,Gravel,0,0,1,0
3,0011165.jpg_Sugar,NaN,0011165.jpg,Sugar,0,0,0,1
4,002be4f.jpg_Fish,233813 878 235213 878 236613 878 238010 881 23...,002be4f.jpg,Fish,1,0,0,0


In [4]:
# Creating training samples

image_col = np.array(df['Image'])
image_files = image_col[::4]
all_labels = df['Class']
X_train, y_train = image_files, all_labels
train_pairs = np.array(list(zip(X_train, y_train)))
TILE_FILENAME_FORMAT = "{triplet_id:05d}_{tile_type}.png"
NUM_TRAIN_SAMPLES = len(train_pairs)
train_samples = train_pairs[np.random.choice(train_pairs.shape[0], NUM_TRAIN_SAMPLES, replace=False), :]

/home/fbrient/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Creating dataset of rectangles divided in tiles.

In [5]:

labs = []
shown = True
t = 0
pathout = DATASETS_path+ 'ZOONIVERSE_DIVIDED/'
for sample in tqdm(train_samples):
        if not shown:
            fig, ax = plt.subplots(figsize=(15, 10))
        img_path = os.path.join(DATASET_DIR, 'train_images', sample[0])
        img = cv2.imread(img_path, 1)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        #plt.imshow(img)
        #plt.show()
        # Get annotations
        labels = df[df['Image_Label'].str.contains(sample[0])]['EncodedPixels']

        patches = []
        for idx, rle in enumerate(labels.values):
            if rle is not np.nan:
                mask = rle2mask(rle)
                #maskn = mask.copy()
                #mask = np.ascontiguousarray(mask , dtype=np.uint8)
                contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                #print("Contours : "+str(contours))
                rgbImg = extract_contour(img, contours, shown=shown)
                rgbImg = cv2.cvtColor(rgbImg, cv2.COLOR_BGR2RGB)
                for x in range(0, rgbImg.shape[0], 256):
                    for y in range(0, rgbImg.shape[1], 256):
                        if rgbImg[x:x+256,y:y+256].shape != (256, 256, 3) or [0,0,0] in rgbImg[x:x+256,y:y+256]:
                            continue
                        
                        tm = t
                        out_name = TILE_FILENAME_FORMAT.format(triplet_id=tm,tile_type='anchor')
                        
                        out_name = pathout+'train/'+out_name
                        cv2.imwrite(out_name,rgbImg[x:x+256,y:y+256])
                        t += 1
                        labs.append(idx)
                if not shown:
                    for contour in contours:
                        poly_patch = Polygon(contour.reshape(-1, 2), closed=True, linewidth=2, edgecolor=COLORS[idx], facecolor=COLORS[idx], fill=True)
                        patches.append(poly_patch)
        if not shown:
            shown = True
            p = PatchCollection(patches, match_original=True, cmap=matplotlib.cm.jet, alpha=0.3)

            ax.imshow(img/255)
            ax.set_title('{} - ({})'.format(sample[0], ', '.join(sample[1].astype(np.str))))
            ax.add_collection(p)
            ax.set_xticklabels([])
            ax.set_yticklabels([])
            plt.show()
with open(pathout+'labels.pickle', 'wb') as handle:
    pickle.dump(labs, handle, protocol=pickle.HIGHEST_PROTOCOL)


/home/fbrient/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


  0%|          | 0/5546 [00:00<?, ?it/s]

# Creating division with rectangles resized as images

In [6]:

labs_resized = []
shown = False
t = 0
pathout = DATASETS_path+ 'ZOONIVERSE_DIVIDED/'

for sample in tqdm(train_samples):
        img_path = os.path.join(DATASET_DIR, 'train_images', sample[0])
        img = cv2.imread(img_path, 1)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        labels = df[df['Image_Label'].str.contains(sample[0])]['EncodedPixels']

        patches = []
        for idx, rle in enumerate(labels.values):
            if rle is not np.nan:
                mask = rle2mask(rle)
                contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                rgbImg = extract_contour(img, contours, shown=False)
                rgbImg = cv2.resize(rgbImg, (256, 256), interpolation = cv2.INTER_AREA)
                rgbImg = cv2.cvtColor(rgbImg, cv2.COLOR_BGR2RGB)
                labs_resized.append(idx)
                tm = t
                out_name = TILE_FILENAME_FORMAT.format(triplet_id=tm,tile_type='anchor')                
                out_name = pathout+'train/'+out_name
                cv2.imwrite(out_name, rgbImg)
                t += 1
with open(pathout+'labels.pickle', 'wb') as handle:
    pickle.dump(labs_resized, handle, protocol=pickle.HIGHEST_PROTOCOL)

/home/fbrient/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


  0%|          | 0/5546 [00:00<?, ?it/s]

# Creating training dataset for the model.

In [7]:
import time
# 607
train_saving_path = DATASETS_path+'ZOONIVERSE_TRAIN/train/'
study_saving_path = DATASETS_path+'ZOONIVERSE_TRAIN/study/'
saving_path = train_saving_path
PI = 3.1456
t = 0
index_img = 0
for sample in tqdm(train_samples):
    if index_img < 0.81*train_samples.shape[0]:
        saving_path = train_saving_path
    else:
        saving_path = study_saving_path
    img_path = os.path.join(DATASET_DIR, 'train_images', sample[0])
    img = cv2.imread(img_path, 1)
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Get annotations
    labels = df[df['Image_Label'].str.contains(sample[0])]['EncodedPixels']

    for x in range(0, img.shape[0], 256):
        for y in range(0, img.shape[1], 256):
            if img[x:x+256,y:y+256].shape != (256, 256, 3):# or  [0,0,0] in rgbImg[x:x+256,y:y+256]:
                continue
            start = time.time()
            time.perf_counter()            
                
            found = False # Variable meaning that we didn't find suiting neighobr tile
            best_neighbor = 0
            best_neighbor_img = []
            while not found: # Getting a neighbor image
                R = 256/2
                theta = random.random() * 2 * PI

                
                x_shift, y_shift = int(R * cos(theta)), int(R * sin(theta))
                neighobr_img = img[x+x_shift:x+x_shift+256,y+y_shift:y+y_shift+256]
                elapsed = time.time() - start
                if len(np.unique(neighobr_img.reshape(-1, neighobr_img.shape[2]), axis=0))>best_neighbor and neighobr_img.shape == (256, 256, 3):
                    best_neighbor = len(np.unique(neighobr_img.reshape(-1, neighobr_img.shape[2]), axis=0))
                    best_neighbor_img = neighobr_img

                if (not (neighobr_img.shape != (256, 256, 3) or len(np.unique(neighobr_img.reshape(-1, neighobr_img.shape[2]), axis=0))<30)) or elapsed>30:
                    found = True
            if [0, 0, 0] in best_neighbor_img:
                continue
            neighobr_img = best_neighbor_img
            found = False # distant

            while not found:
                random_img = train_samples[random.randint(0, train_samples.shape[0]-1)]
                if random_img[0] == sample[0]:
                    continue
                distant_img_path = os.path.join(DATASET_DIR, 'train_images', random_img[0])
                distant_img = cv2.imread(distant_img_path, 1)
                #distant_img = cv2.cvtColor(distant_img, cv2.COLOR_BGR2RGB)
                x_distant = random.randint(0, distant_img.shape[0]-257)
                y_distant = random.randint(0, distant_img.shape[1]-257)
                distant_img_shaped = distant_img[x_distant:x_distant+256,y_distant:y_distant+256]
                if not (distant_img_shaped.shape != (256, 256, 3) or [0,0,0] in distant_img_shaped):
                    found = True
                
            tm = t
            
            # anchor
            out_name = TILE_FILENAME_FORMAT.format(triplet_id=tm,tile_type='anchor')
            out_name = saving_path + out_name
            cv2.imwrite(out_name, img[x:x+256,y:y+256])

            # neighbor
            out_name_neighbor = TILE_FILENAME_FORMAT.format(triplet_id=tm,tile_type='neighbor')
            out_name_neighbor = saving_path + out_name_neighbor
            cv2.imwrite(out_name_neighbor, neighobr_img)

            # distant
            out_name_distant = TILE_FILENAME_FORMAT.format(triplet_id=tm,tile_type='distant')
            out_name_distant = saving_path + out_name_distant
            cv2.imwrite(out_name_distant, distant_img_shaped)

            t += 1
    index_img += 1

/home/fbrient/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


  0%|          | 0/5546 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [8]:
import kornia
print(kornia.__version__)

0.5.11


/home/fbrient/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
import torch

torch.cuda.is_available()

/home/fbrient/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


False

# Evaluating dataset threshold

In [11]:

labs = []
shown = True
t = 0
stats = {}
for sample in tqdm(train_samples):
        if not shown:
            fig, ax = plt.subplots(figsize=(15, 10))
        img_path = os.path.join(DATASET_DIR, 'train_images', sample[0])
        img = cv2.imread(img_path, 1)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        #plt.imshow(img)
        #plt.show()
        # Get annotations
        labels = df[df['Image_Label'].str.contains(sample[0])]['EncodedPixels']
        #print("labels : "+str(labels.values))
        patches = []
        dominance = 0
        recapitulatif = {}
        all_size = (0, 0, 0)
        for idx, rle in enumerate(labels.values):
            if rle is not np.nan:
                mask = rle2mask(rle)
                #maskn = mask.copy()
                #mask = np.ascontiguousarray(mask , dtype=np.uint8)
                contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                #print("Contours : "+str(contours))
                rgbImages = extract_contour(img, contours, shown=shown, direct=False)
                """
                print("IDX : "+str(idx))
                print("len : "+str(len(rgbImages)))
                print("first : "+str(rgbImages[0].shape))
                """
                summed = (0, 0)
                for rgbImg in rgbImages:
                    summed = tuple(map(operator.add, summed, rgbImg.shape))
                    #print(type(rgbImg.shape))
                #print("mean of class : "+str(summed)+"\n")
                all_size = (all_size[0]+summed[0], all_size[1]+summed[1]) #tuple(map(operator.add, all_size, summed))
                recapitulatif[str(idx)] = (summed[0],summed[1])
                               
                """
                for rgbImg in rgbImages:
                    print(rgbImg.shape)
                """
                #break
                #rgbImg = cv2.cvtColor(rgbImg, cv2.COLOR_BGR2RGB)

                for x in range(0, rgbImg.shape[0], 256):
                    for y in range(0, rgbImg.shape[1], 256):
                        if rgbImg[x:x+256,y:y+256].shape != (256, 256, 3) or [0,0,0] in rgbImg[x:x+256,y:y+256]:
                            continue
                        
                        tm = t
                        out_name = TILE_FILENAME_FORMAT.format(triplet_id=tm,tile_type='anchor')
                        
                        out_name = '../NC/division/train/'+out_name
                        cv2.imwrite(out_name,rgbImg[x:x+256,y:y+256])
                        t += 1
                        labs.append(idx)
                if not shown:
                    for contour in contours:
                        #print("contour : "+str(contour))
                        poly_patch = Polygon(contour.reshape(-1, 2), closed=True, linewidth=2, edgecolor=COLORS[idx], facecolor=COLORS[idx], fill=True)
                        patches.append(poly_patch)
                
        if not shown:
            shown = True
            p = PatchCollection(patches, match_original=True, cmap=matplotlib.cm.jet, alpha=0.3)

            ax.imshow(img/255)
            ax.set_title('{} - ({})'.format(sample[0], ', '.join(sample[1].astype(np.str))))
            ax.add_collection(p)
            ax.set_xticklabels([])
            ax.set_yticklabels([])
            plt.show()
        
        """
        print("\n\nlast all size : "+str(all_size))
        #all_size = all_size[0]*all_size[1]
        """
        recapitulatif_percentage = {}
        #print("RECAPITULATIF X Y : "+str(recapitulatif))
        for key in recapitulatif:
            recapitulatif_percentage[key] = (recapitulatif[key][0]/all_size[0] + recapitulatif[key][1]/all_size[1])/2
        best = 0
        best_idx = -1
        for key in recapitulatif_percentage:
            if recapitulatif_percentage[key]>best:
                best = recapitulatif_percentage[key]
        try:
            stats[round(best, 2)] += 1
        except:
            stats[round(best, 2)] = 1
        """
        print("\nRECAPITULATIF : "+str(recapitulatif))
        print("RECAPITULATIF PERCENTAGE : "+str(recapitulatif_percentage))
        print("SUMMED : "+str(all_size))
        break
        """

/home/fbrient/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


  0%|          | 0/5546 [00:00<?, ?it/s]

In [ ]:
import seaborn as sns
sns.set(rc={'figure.figsize':(35, 10)})
sns.set_theme(style="whitegrid")
fig, ax = plt.subplots()
keys = list(stats.keys())
# get values in the same order as keys, and parse percentage values
#vals = [float(stats[k][:-1]) for k in keys]
sns.barplot(x=keys, y=list(stats.values()), ax=ax)
ax.set_ylabel('Number of images',fontsize=40)
ax.set_xlabel('Best Coverage Percentage',fontsize=40)
#ax.xticks(fontsize=14, rotation=90)
plt.yticks(fontsize=30)
plt.title("Plot of coverage percentage Zooniverse Dataset")
plt.savefig('coverage.png')


In [ ]:
import collections
stacked = 0
stacked_stats = {}
stats = collections.OrderedDict(sorted(stats.items()))
for key in stats:
    stacked += stats[key]
    stacked_stats[key] = stacked
    #print(stacked)
print(stacked_stats)

fig, ax = plt.subplots()
sns.set(rc={'figure.figsize':(35, 40)})
sns.set_theme(style="whitegrid")
keys_stacked = list(stacked_stats.keys())
# get values in the same order as keys, and parse percentage values
#vals = [float(stats[k][:-1]) for k in keys]
sns.barplot(x=keys_stacked, y=list(stacked_stats.values()), ax=ax)
ax.set_ylabel('Number of images',fontsize=40)
ax.set_xlabel('Best Coverage Percentage',fontsize=40)
#ax.xticks(fontsize=14, rotation=90)
plt.yticks(fontsize=30)
plt.title("Cumulative plot of coverage percentage Zooniverse Dataset")
plt.savefig('cumulative_coverage.png')

# Distribution couverture des classes / shape image

In [ ]:
import collections

labs = []
shown = True
t = 0
stats = {}
index_distributions = {
    '0':{},
    '1':{},
    '2':{},
    '3':{},
}
for i in range(0, 101):
    index_distributions['0'][str(i/100)] = 0
    index_distributions['1'][str(i/100)] = 0
    index_distributions['2'][str(i/100)] = 0
    index_distributions['3'][str(i/100)] = 0

for sample in tqdm(train_samples):
        if not shown:
            fig, ax = plt.subplots(figsize=(15, 10))
        img_path = os.path.join(DATASET_DIR, 'train_images', sample[0])
        img = cv2.imread(img_path, 1)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        #plt.imshow(img)
        #plt.show()
        # Get annotations
        labels = df[df['Image_Label'].str.contains(sample[0])]['EncodedPixels']
        #print("labels : "+str(labels.values))
        patches = []
        dominance = 0
        recapitulatif = {}
        all_size = (0, 0, 0)
        for idx, rle in enumerate(labels.values):
            if rle is not np.nan:
                mask = rle2mask(rle)
                contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                #print("Contours : "+str(contours))
                
                img_copy = np.zeros(img.shape)
                cnts = sorted(contours, key=cv2.contourArea, reverse=True)
                rois = []
                # Find bounding box and extract ROI
                for c in cnts:
                    x,y,w,h = cv2.boundingRect(c)
                    img_copy[y:y+h, x:x+w] = (1, 1, 1)
                percentage = np.count_nonzero((img_copy==(1, 1, 1)).all(axis = 2)) / ( img_copy.shape[0]*img_copy.shape[1] )
                percentage_round = round(percentage, 2)
                index_distributions[str(idx)][str(percentage_round)] += 1
                """
                print("sample : "+str(img_path))
                print("idx : "+str(idx))
                print("% pixels "+str(  ) )
                """


In [ ]:
associations = ['Fish', 'Flower', 'Gravel', 'Sugar']
sns.set(rc={'figure.figsize':(60, 20)})
sns.set_theme(style="whitegrid")
for classe in index_distributions:
    fig, ax = plt.subplots()    
    keys_stacked = list(index_distributions[classe].keys())
    # get values in the same order as keys, and parse percentage values
    #vals = [float(stats[k][:-1]) for k in keys]
    sns.barplot(x=keys_stacked, y=list(index_distributions[classe].values()), ax=ax)
    ax.set_ylabel('Number of images',fontsize=40)
    ax.set_xlabel('Coverage Percentage',fontsize=40)
    #ax.xticks(fontsize=14, rotation=90)
    plt.yticks(fontsize=30)
    plt.figtext(.5,.9,str(associations[int(classe)])+" Coverage % on Images", fontsize=100, ha='center')
    plt.savefig(str(associations[int(classe)])+'_coverage.png')

# Counting average shape of rectangles for each class

In [ ]:
import collections

labs = []
shown = True
t = 0
stats = {}
index_distributions = {
    '0':{'x':{}, 'y':{}},
    '1':{'x':{}, 'y':{}},
    '2':{'x':{}, 'y':{}},
    '3':{'x':{}, 'y':{}},
}

for i in range(0, 2101):
    index_distributions['0']['x'][str(i)] = 0
    index_distributions['0']['y'][str(i)] = 0

    index_distributions['1']['x'][str(i)] = 0
    index_distributions['1']['y'][str(i)] = 0

    index_distributions['2']['x'][str(i)] = 0
    index_distributions['2']['y'][str(i)] = 0

    index_distributions['3']['x'][str(i)] = 0
    index_distributions['3']['y'][str(i)] = 0

for sample in tqdm(train_samples):
        if not shown:
            fig, ax = plt.subplots(figsize=(15, 10))
        img_path = os.path.join(DATASET_DIR, 'train_images', sample[0])
        img = cv2.imread(img_path, 1)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        #plt.imshow(img)
        #plt.show()
        # Get annotations
        labels = df[df['Image_Label'].str.contains(sample[0])]['EncodedPixels']
        #print("labels : "+str(labels.values))
        patches = []
        dominance = 0
        recapitulatif = {}
        all_size = (0, 0, 0)
        for idx, rle in enumerate(labels.values):
            if rle is not np.nan:
                mask = rle2mask(rle)
                contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                #print("Contours : "+str(contours))
                
                img_copy = np.zeros(img.shape)
                cnts = sorted(contours, key=cv2.contourArea, reverse=True)
                rois = []
                # Find bounding box and extract ROI
                for c in cnts:
                    x,y,w,h = cv2.boundingRect(c)
                    img_copy[y:y+h, x:x+w] = (1, 1, 1)
                    index_distributions[str(idx)]['x'][str(h)] += 1
                    index_distributions[str(idx)]['y'][str(w)] += 1
                    break


In [ ]:
print(index_distributions['1']['x'])

In [ ]:
associations = ['Fish', 'Flower', 'Gravel', 'Sugar']
sns.set(rc={'figure.figsize':(600, 20)})
sns.set_theme(style="whitegrid")
for classe in index_distributions:
    fig, ax = plt.subplots()    
    keys_stacked = list(index_distributions[classe]['x'].keys())
    # get values in the same order as keys, and parse percentage values
    #vals = [float(stats[k][:-1]) for k in keys]
    sns.barplot(x=keys_stacked, y=list(index_distributions[classe]['x'].values()), ax=ax)
    ax.set_ylabel('Number of images',fontsize=40)
    ax.set_xlabel('Coverage Percentage',fontsize=40)
    #ax.xticks(fontsize=14, rotation=90)
    plt.yticks(fontsize=30)
    plt.figtext(.5,.9,str(associations[int(classe)])+" Coverage % on Images", fontsize=100, ha='center')
    plt.savefig(str(associations[int(classe)])+'_size_shape_x_coverage.png')

In [ ]:
associations = ['Fish', 'Flower', 'Gravel', 'Sugar']
sns.set(rc={'figure.figsize':(600, 20)})
sns.set_theme(style="whitegrid")
for classe in index_distributions:
    fig, ax = plt.subplots()    
    keys_stacked = list(index_distributions[classe]['y'].keys())
    # get values in the same order as keys, and parse percentage values
    #vals = [float(stats[k][:-1]) for k in keys]
    sns.barplot(x=keys_stacked, y=list(index_distributions[classe]['y'].values()), ax=ax)
    ax.set_ylabel('Number of images',fontsize=40)
    ax.set_xlabel('Coverage Percentage',fontsize=40)
    #ax.xticks(fontsize=14, rotation=90)
    plt.yticks(fontsize=30)
    plt.figtext(.5,.9,str(associations[int(classe)])+" Coverage % on Images", fontsize=100, ha='center')
    plt.savefig(str(associations[int(classe)])+'_size_shape_y_coverage.png')

# Creation du jeu de données pour l'entrainement de la classification

In [ ]:
labs = []
shown = True
t = 0
associations = ['Fish', 'Flower', 'Gravel', 'Sugar']
PATH_TRAIN = DATASETS_path+"CLASSIF_ALL_IMAGE/"
count_class = {}; count_class["Fish"] = 0; count_class["Flower"] = 0; count_class["Gravel"] = 0; count_class["Sugar"] = 0;
stats = {}
for sample in tqdm(train_samples):
        if not shown:
            fig, ax = plt.subplots(figsize=(15, 10))
        img_path = os.path.join(DATASET_DIR, 'train_images', sample[0])
        img = cv2.imread(img_path, 1)
        labels = df[df['Image_Label'].str.contains(sample[0])]['EncodedPixels']
        patches = []
        dominance = 0
        recapitulatif = {}
        all_size = (0, 0, 0)
        for idx, rle in enumerate(labels.values):
            if rle is not np.nan:
                mask = rle2mask(rle)
                contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                rgbImages = extract_contour(img, contours, shown=shown, direct=False)
                summed = (0, 0)
                for rgbImg in rgbImages:
                    summed = tuple(map(operator.add, summed, rgbImg.shape))
                all_size = (all_size[0]+summed[0], all_size[1]+summed[1])
                recapitulatif[str(idx)] = (summed[0],summed[1])
                               
            recapitulatif_percentage = {}
        for key in recapitulatif:
            recapitulatif_percentage[key] = (recapitulatif[key][0]/all_size[0] + recapitulatif[key][1]/all_size[1])/2
        best = 0
        best_idx = -1
        for key in recapitulatif_percentage:
            if recapitulatif_percentage[key]>best:
                best = recapitulatif_percentage[key]
                best_idx = key
        if best>0.64:
            out_name = PATH_TRAIN+"/"+associations[int(best_idx)]+"/"+str(count_class[str(associations[int(best_idx)])])+".jpg"
            #print("out name : "+str(out_name))                      
            cv2.imwrite(out_name, img)
            count_class[str(associations[int(best_idx)])] += 1
            


# Resized classidying dataset

In [ ]:
labs = []
shown = True
t = 0
associations = ['Fish', 'Flower', 'Gravel', 'Sugar']
PATH_TRAIN = DATASETS_path+"CLASSIF_RESIZED/"
count_class = {}; count_class["Fish"] = 0; count_class["Flower"] = 0; count_class["Gravel"] = 0; count_class["Sugar"] = 0;
stats = {}
for sample in tqdm(train_samples):
        img_path = os.path.join(DATASET_DIR, 'train_images', sample[0])
        img = cv2.imread(img_path, 1)
        labels = df[df['Image_Label'].str.contains(sample[0])]['EncodedPixels']
        for idx, rle in enumerate(labels.values):
            if rle is not np.nan:
                mask = rle2mask(rle)
                contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                rgbImg = extract_contour(img, contours, shown=shown, direct=True)

                rgbImg = cv2.resize(rgbImg, (256, 256), interpolation = cv2.INTER_AREA)
                #rgbImg = cv2.cvtColor(rgbImg, cv2.COLOR_BGR2RGB)              
                out_name = PATH_TRAIN+str(associations[int(idx)])+"/"+str(count_class[associations[int(idx)]])+".jpeg"
                cv2.imwrite(out_name, rgbImg)
                count_class[associations[int(idx)]] += 1


# Biggest square classifying dataset

In [ ]:
labs = []
shown = True
t = 0
associations = ['Fish', 'Flower', 'Gravel', 'Sugar','Undefined']
PATH_TRAIN = DATASETS_path+"CLASSIF_BIGGEST_SQUARE/"
count_class = {}; count_class["Fish"] = 0; count_class["Flower"] = 0; count_class["Gravel"] = 0; count_class["Sugar"] = 0;
count_class["Undefined"] = 0
stats = {}
for sample in tqdm(train_samples):
        img_path = os.path.join(DATASET_DIR, 'train_images', sample[0])
        img = cv2.imread(img_path, 1)
        labels = df[df['Image_Label'].str.contains(sample[0])]['EncodedPixels']
        cant = []
        for idx, rle in enumerate(labels.values):
            if rle is not np.nan:
                mask = rle2mask(rle)
                contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                cant.append(contours)
                rgbImg = extract_contour(img, contours, shown=shown, direct=True)
                min_shape = min(rgbImg.shape[0], rgbImg.shape[1])
                if rgbImg.shape[0]>=128:
                    rgbImg = cv2.resize(rgbImg[0:min_shape, 0:min_shape], (256, 256), interpolation = cv2.INTER_AREA)
                    labs.append(idx)
                    #rgbImg = cv2.cvtColor(rgbImg, cv2.COLOR_BGR2RGB)              
                    out_name = PATH_TRAIN+str(associations[int(idx)])+"/"+str(count_class[associations[int(idx)]])+".jpeg"
                    cv2.imwrite(out_name, rgbImg)
                    count_class[associations[int(idx)]] += 1
        for contours in cant:
            for c in contours:
                img = cv2.drawContours(img, [c], -1, (0,0,0), -1)

        undef = extract_non_black(img)
        undef = cv2.resize(undef, (256, 256), interpolation = cv2.INTER_AREA)
        if [0,0,0] not in undef and undef.shape[0]>=128:
            idx = 4
            labs.append(idx)
            #rgbImg = cv2.cvtColor(rgbImg, cv2.COLOR_BGR2RGB)              
            out_name = PATH_TRAIN+str(associations[int(idx)])+"/"+str(count_class[associations[int(idx)]])+".jpeg"
            cv2.imwrite(out_name, undef)
            count_class[associations[int(idx)]] += 1


In [ ]:

with open(PATH_TRAIN+'labels.pickle', 'wb') as handle:
    pickle.dump(labs, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Biggest Square clustering dataset

In [ ]:

labs = []
shown = False
t = 0
patches = []
associations = ['Fish', 'Flower', 'Gravel', 'Sugar','Undefined']
PATH_TRAIN = DATASETS_path+"CLUSTERING_BIGGEST_SQUARE/train/"
count_class = {}; count_class["Fish"] = 0; count_class["Flower"] = 0; count_class["Gravel"] = 0; count_class["Sugar"] = 0; 
count_class["Undefined"] = 0
stats = {}
a = 0
print("LEN : "+str(len(train_samples)))
for sample in tqdm(train_samples):
        img_path = os.path.join(DATASET_DIR, 'train_images', sample[0])
        img = cv2.imread(img_path, 1)
        #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
        labels = df[df['Image_Label'].str.contains(sample[0])]['EncodedPixels']
        patches = []
        for idx, rle in enumerate(labels.values):
            if rle is not np.nan:
                mask = rle2mask(rle)
                contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                rgbImg = extract_contour(img, contours, shown=shown, direct=True)
                min_shape = min(rgbImg.shape[0], rgbImg.shape[1])
                
                
                rgbImg = cv2.resize(rgbImg[0:min_shape, 0:min_shape], (256, 256), interpolation = cv2.INTER_AREA)
                if [0,0,0] not in rgbImg and min_shape>=128:
                    labs.append(idx)
                                
                    tm = t
                    out_name = TILE_FILENAME_FORMAT.format(triplet_id=tm,tile_type='anchor')
                    
                    out_name = PATH_TRAIN+out_name

                    cv2.imwrite(out_name, rgbImg)
                    count_class[associations[int(idx)]] += 1
                    t += 1
            
            for c in contours:
                img = cv2.drawContours(img, [c], -1, (0,0,0), -1)

        undef = extract_non_black(img)
        undef = cv2.resize(undef, (256, 256), interpolation = cv2.INTER_AREA)
        
        if [0,0,0] not in undef and undef.shape[0]>=128:
            tm = t
            out_name = TILE_FILENAME_FORMAT.format(triplet_id=tm,tile_type='anchor')
            """
            fig = plt.figure()
            plt.imshow(undef)
            fig.suptitle(out_name, fontsize=20)
            plt.show()
            """
            labs.append(4)
            out_name = PATH_TRAIN+out_name
            cv2.imwrite(out_name, undef)
            count_class[associations[int(4)]] += 1
            t += 1

with open(DATASETS_path+"CLUSTERING_BIGGEST_SQUARE/labels.pickle", 'wb') as handle:
    pickle.dump(labs, handle, protocol=pickle.HIGHEST_PROTOCOL)


# Multi-Scale generation

In [ ]:
#divisions = []
from pathlib import Path
import pathlib
divisions = [64, 128, 256, 512, 612, 850, 1024 ] #range(416, 1024, 32)#
divisions = [612]
shown = False
for div in divisions:
    print("DIV : "+str(div))
    labs = []
    shown = True
    t = 0
    associations = ['Fish', 'Flower', 'Gravel', 'Sugar']
    PATH_TRAIN = DATASETS_path+"MULTI_SCALE/"+str(div)+"/train/"
    if not Path(PATH_TRAIN).is_dir():
        p = pathlib.Path(PATH_TRAIN)
        p.mkdir(parents=True, exist_ok=True)
    count_class = {}; count_class["Fish"] = 0; count_class["Flower"] = 0; count_class["Gravel"] = 0; count_class["Sugar"] = 0;
    stats = {}
    for sample in tqdm(train_samples):
            img_path = os.path.join(DATASET_DIR, 'train_images', sample[0])
            img = cv2.imread(img_path, 1)
            labels = df[df['Image_Label'].str.contains(sample[0])]['EncodedPixels']
            for idx, rle in enumerate(labels.values):
                if rle is not np.nan:
                    mask = rle2mask(rle)
                    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                    rgbImg = extract_contour(img, contours, shown=shown, direct=True)

                    for x in range(0, rgbImg.shape[0], div):
                        for y in range(0, rgbImg.shape[1], div):
                            if rgbImg[x:x+div,y:y+div].shape != (div, div, 3) or [0,0,0] in rgbImg[x:x+div,y:y+div]:
                                continue
                            tm = t
                            if t == 10000:
                                break
                            grgr = cv2.resize(rgbImg[x:x+div,y:y+div], (256, 256), interpolation = cv2.INTER_AREA)
                            out_name = TILE_FILENAME_FORMAT.format(triplet_id=tm,tile_type='anchor')
                            
                            out_name = PATH_TRAIN+out_name
                            cv2.imwrite(out_name, grgr)
                            t += 1
                            labs.append(idx)
                    
            if t == 10000:
                break
    with open(DATASETS_path+"MULTI_SCALE/"+str(div)+"/labels.pickle", 'wb') as handle:
        pickle.dump(labs, handle, protocol=pickle.HIGHEST_PROTOCOL)
